In [1]:
import tensorflow as tf
import os

In [2]:
_URL = 'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'

zip_file = tf.keras.utils.get_file(origin=_URL, fname='flower_photos.tgz', extract=True, cache_subdir='/content')

base_dir = os.path.join(os.path.dirname(zip_file), 'flower_photos')

228818944/228813984 [==============================] - 4s 0us/step


In [3]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(base_dir,
                                              target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                              batch_size=BATCH_SIZE,
                                              subset='training')

val_generator = datagen.flow_from_directory(base_dir,
                                              target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                              batch_size=BATCH_SIZE,
                                              subset='validation')


Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [5]:
print(train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}


In [6]:
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

base_model =  tf.keras.applications.MobileNetV2(input_shape=IMAGE_SHAPE,
                                                include_top=False, 
                                                weights='imagenet')

9412608/9406464 [==============================] - 0s 0us/step


In [7]:
base_model.trainable = False

In [8]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(5, activation='softmax')
])

In [9]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
epochs = 10

history = model.fit(train_generator, 
                    epochs=epochs,
                    validation_data=val_generator)

Epoch 1/10
46/46 [==============================] - 21s 284ms/step - loss: 0.9861 - accuracy: 0.6750 - val_loss: 0.4247 - val_accuracy: 0.8495
Epoch 2/10
46/46 [==============================] - 12s 255ms/step - loss: 0.2436 - accuracy: 0.9198 - val_loss: 0.4241 - val_accuracy: 0.8482
Epoch 3/10
46/46 [==============================] - 12s 255ms/step - loss: 0.1481 - accuracy: 0.9486 - val_loss: 0.4234 - val_accuracy: 0.8646
Epoch 4/10
46/46 [==============================] - 12s 255ms/step - loss: 0.0912 - accuracy: 0.9715 - val_loss: 0.4395 - val_accuracy: 0.8605
Epoch 5/10
46/46 [==============================] - 12s 255ms/step - loss: 0.0557 - accuracy: 0.9849 - val_loss: 0.4579 - val_accuracy: 0.8632
Epoch 6/10
46/46 [==============================] - 12s 258ms/step - loss: 0.0294 - accuracy: 0.9979 - val_loss: 0.5024 - val_accuracy: 0.8646
Epoch 7/10
46/46 [==============================] - 12s 257ms/step - loss: 0.0160 - accuracy: 0.9995 - val_loss: 0.5539 - val_accuracy: 0.8591

In [11]:
saved_model_dir = ''
tf.saved_model.save(model, 
                    saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()


with open('flower_classifier_model.tflite', 'wb') as file:
  file.write(tflite_model)

INFO:tensorflow:Assets written to: assets


In [13]:
from google.colab import files

files.download('flower_classifier_model.tflite')
files.download('labels.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>